In [1]:
import sys, torch
import numpy as np
from DP.label_model import *
from DP.synthetic import SyntheticDataGenerator
from DP.params import generate_temporal_params

In [2]:
device = torch.device('cpu')

In [3]:
T=4
m_per_task=3
feasible_y = np.array(np.meshgrid(*[[-1,1]]*T)).T.reshape(-1,T)
class_balance = torch.FloatTensor([np.prod([0.9*(y[t]==y[t+1])+0.1*(y[t]!=y[t+1]) for t in range(T-1)]) \
                                   for y in feasible_y]).to(device)*0.5
params = generate_temporal_params(m_per_task=m_per_task, T=T, device=device)

In [4]:
data_generator = SyntheticDataGenerator(m=params['m'], 
                                        T=params['T'], 
                                        edges=params['edges'],
                                        coverage_sets=params['coverage'],
                                        mu_sharing=params['sharing']['mu'],
                                        phi_sharing=params['sharing']['phi'],
                                        mu_list = params['mu'],
                                        phi_list = params['phi'],
                                        alpha_list = params['alpha'],
                                        device=device,
                                        class_balance=class_balance)
train_data = data_generator.generate_data(n=50000, seed=0)
test_data =  data_generator.generate_data(n=5000, seed=1)

In [5]:
markov_model = DPLabelModel(m=params['m'], 
                            T=params['T'], 
                            edges=params['edges'],
                            coverage_sets=params['coverage'],
                            mu_sharing=params['sharing']['mu'],
                            phi_sharing=params['sharing']['phi'],
                            device=device,
                            class_balance=class_balance)

In [6]:
%%time
optimize(markov_model, train_data['O_hat'], 
         num_iter=1000, lr=1e-3, momentum=0.85, clamp=True);

iteration=0 loss=446.1423645019531
iteration=100 loss=6.457624435424805
iteration=200 loss=0.01432060170918703
iteration=300 loss=0.010976724326610565
iteration=400 loss=0.010897819884121418
iteration=500 loss=0.01089534442871809
iteration=600 loss=0.010894772596657276
iteration=700 loss=0.01089426688849926
iteration=800 loss=0.010893801227211952
iteration=900 loss=0.010893355123698711
iteration=999 loss=0.010892949998378754
CPU times: user 1min 17s, sys: 699 ms, total: 1min 18s
Wall time: 1min 23s


In [7]:
compare_parameters(markov_model, params['mu'], params['phi'], params['alpha'])

lambdas 0, 3, 6, 9
learned_mu=['0.961', '0.039']	mu=['0.960', '0.040']
           ['0.038', '0.961']	   ['0.040', '0.960']
lambdas 1, 4, 7, 10
learned_mu=['0.920', '0.079']	mu=['0.920', '0.080']
           ['0.080', '0.921']	   ['0.080', '0.920']
lambdas 2, 5, 8, 11
learned_mu=['0.880', '0.120']	mu=['0.880', '0.120']
           ['0.120', '0.880']	   ['0.120', '0.880']

Learned Phi for (0, 3), (3, 6), (6, 9)
[[[0.943 0.038 0.039 0.019]
  [0.019 0.923 0.    0.02 ]]

 [[0.019 0.001 0.922 0.02 ]
  [0.02  0.038 0.039 0.941]]]
Phi for (0, 3), (3, 6), (6, 9)
[[[0.94  0.038 0.038 0.02 ]
  [0.02  0.922 0.002 0.02 ]]

 [[0.02  0.002 0.922 0.02 ]
  [0.02  0.038 0.038 0.94 ]]]
Learned Phi for (1, 4), (4, 7), (7, 10)
[[[0.881 0.074 0.076 0.039]
  [0.039 0.846 0.002 0.04 ]]

 [[0.039 0.005 0.844 0.039]
  [0.04  0.075 0.077 0.882]]]
Phi for (1, 4), (4, 7), (7, 10)
[[[0.88  0.074 0.074 0.04 ]
  [0.04  0.846 0.006 0.04 ]]

 [[0.04  0.006 0.846 0.04 ]
  [0.04  0.074 0.074 0.88 ]]]
Learned Phi for (2, 5)

In [8]:
R_predictions = markov_model.predict(test_data['Li'])